# 기본 정리

## Library

In [1]:
# Import Library
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontManager
import os
import pandas as pd
import numpy as np
from urllib.parse import urlparse, parse_qs, unquote, urlunparse, urlencode
import json
import re
from IPython.core.interactiveshell import InteractiveShell 
import urllib.parse
import requests
from bs4 import BeautifulSoup
import hashlib
import validators 
import nbimporter
from datetime import datetime, timedelta
import math



## Directory of Files

In [2]:
### 현재 작업 디렉토리를 가져오기
current_directory = os.getcwd()

### downloads 상대경로를 절대경로로 변환
relative_path_downloads = r'..\\..\\..\\Downloads'
download_path = os.path.join(current_directory, relative_path_downloads)

### "preprocessed" 상대경로를 절대경로로 변환
preprocessed_path = os.path.join(download_path, 'preprocessed')

### Check if the "preprocessed" folder exists
if os.path.exists(preprocessed_path):
    print(f"The 'preprocessed' folder already exists at: {preprocessed_path}")
else:
    # "preprocessed" 폴더 생성
    os.makedirs(preprocessed_path)


### ContentsList 상대경로를 절대경로로 변환
relative_path_contentslist = r'..\\..\\..\\OneDrive - 클리 주식회사\문서 - 클리주식회사\003_Project-3\07_콘텐츠성과측정\ContentsList.xlsx'
contentslist_path = os.path.join(current_directory, relative_path_contentslist)

### DailyRawdata
path_dailyrawdata = r'..\\..\\..\\OneDrive - 클리 주식회사\문서 - 클리주식회사\003_Project-3\07_콘텐츠성과측정\DailyRawdata'

### 07_콘텐츠성과측정 상대경로를 절대경로로 변환
relative_path_upper_path = r'..\\..\\..\\OneDrive - 클리 주식회사\문서 - 클리주식회사\003_Project-3\07_콘텐츠성과측정'
upper_path = os.path.join(current_directory, relative_path_upper_path)


The 'preprocessed' folder already exists at: c:\Users\ch.kang\Documents\GitHub\funnel_test\..\\..\\..\\Downloads\preprocessed


## Directory of Files_house

### 현재 작업 디렉토리를 가져오기
current_directory = os.getcwd()

### downloads 상대경로를 절대경로로 변환
relative_path_downloads = r'..\\..\\..\\..\\Downloads'
download_path = os.path.join(current_directory, relative_path_downloads)

### "preprocessed" 상대경로를 절대경로로 변환
preprocessed_path = os.path.join(download_path, 'preprocessed')

### Check if the "preprocessed" folder exists
if os.path.exists(preprocessed_path):
    print(f"The 'preprocessed' folder already exists at: {preprocessed_path}")
else:
    # "preprocessed" 폴더 생성
    os.makedirs(preprocessed_path)


### ContentsList 상대경로를 절대경로로 변환
relative_path_contentslist = r'..\\..\\..\\..\\OneDrive - 클리 주식회사\문서 - 클리주식회사\003_Project-3\07_콘텐츠성과측정\ContentsList.xlsx'
contentslist_path = os.path.join(current_directory, relative_path_contentslist)

### DailyRawdata
path_dailyrawdata = r'..\\..\\..\\..\\OneDrive - 클리 주식회사\문서 - 클리주식회사\003_Project-3\07_콘텐츠성과측정\DailyRawdata'

### 07_콘텐츠성과측정 상대경로를 절대경로로 변환
relative_path_upper_path = r'..\\..\\..\\..\\OneDrive - 클리 주식회사\문서 - 클리주식회사\003_Project-3\07_콘텐츠성과측정'
upper_path = os.path.join(current_directory, relative_path_upper_path)




## Import Original Data

In [3]:
# Import Original Data
# 폴더 내의 파일 목록 가져오기
file_list = os.listdir(path_dailyrawdata)

# 'ga'로 시작하는 파일 필터링 및 불러오기
ga_files = [file for file in file_list if file.startswith('ga')]
ga_files = [file for file in ga_files if file != 'ga_backup']

# 'blog'로 시작하는 파일 필터링 및 불러오기
blog_files = [file for file in file_list if file.startswith('blog')]

# contentslist file import
contentslist = pd.read_excel(contentslist_path)

# 빈 데이터프레임을 생성하여 파일을 하나씩 읽어서 합치기
raw_df_ga = pd.DataFrame()
for file in ga_files:
    file_path = os.path.join(path_dailyrawdata, file)  # 파일의 전체 경로 생성
    temp_df = pd.read_excel(file_path, dtype={'아이디': str}) # 파일을 데이터프레임으로 읽기
    raw_df_ga = pd.concat([raw_df_ga, temp_df], ignore_index=True)  # 읽은 데이터프레임을 기존 데이터프레임에 합치기

# 모든 'ga'로 시작하는 파일을 하나의 데이터프레임으로 합친 결과를 raw_df_ga 변수에 저장
    
# 빈 데이터프레임을 생성하여 파일을 하나씩 읽어서 합치기
raw_df_blog = pd.DataFrame()
for file in blog_files:
    file_path = os.path.join(path_dailyrawdata, file)  # 파일의 전체 경로 생성
    temp_df = pd.read_excel(file_path, dtype={'아이디': str}) # 파일을 데이터프레임으로 읽기
    raw_df_blog = pd.concat([raw_df_blog, temp_df], ignore_index=True)  # 읽은 데이터프레임을 기존 데이터프레임에 합치기





In [4]:
df_ga = raw_df_ga.copy()
df_blog = raw_df_blog.copy()

In [5]:
# 합쳐진 파일은 바탕화면에 따로 저장해둠.
# raw_df_ga.to_excel(os.path.join(desktop_path, 'GA.xlsx'), index = False)
# raw_df_blog.to_excel(os.path.join(desktop_path, 'blog.xlsx'), index = False)


In [6]:
print(df_ga.shape)
print(df_blog.shape)

(17373, 13)
(1571, 9)


## Def

In [7]:
def modify_url(url_0, all_keys):
    if url_0 is None or not isinstance(url_0, str):
        return url_0  # Skip if the URL is None or not a string

    # Remove '=' from the beginning
    if url_0.startswith('='):
        url_0 = url_0[1:]
    
    url_0 = url_0.replace('https://m.', 'https://')
    url_0 = url_0.replace('https://s.', 'https://')

    parsed_url = urlparse(url_0)
    scheme = parsed_url.scheme
    netloc = parsed_url.netloc
    path = parsed_url.path
    query = parsed_url.query
    fragment = parsed_url.fragment

    if path.endswith('/PostView.naver') or path.endswith('/PostList.naver') or path.endswith('/FeedList.naver'):
        path = ''

    query_dict = parse_qs(query)

    all_keys.update(query_dict.keys())

    params_to_keep_query = ['blogId', 'logNo', 'bmode', 'idx']  # Include 'bmode' and 'idx'

    for param in params_to_keep_query:
        if param in query_dict:
            query_dict[param] = query_dict[param]

    modified_query = urlencode(query_dict, doseq=True)
    modified_url = urlunparse((scheme, netloc, path, '', modified_query, ''))

    if modified_url.endswith('/'):
        modified_url = modified_url[:-1]

    return modified_url




def extract_korean(text):
    if text is None or not isinstance(text, str):
        return text  # 입력값이 None 또는 문자열이 아닌 경우 그대로 반환

    # '+'를 띄어쓰기로 치환한 후, 모든 한글을 추출하는 정규표현식
    korean_pattern = re.compile('[ㄱ-ㅎㅏ-ㅣ가-힣]+')
    text = text.replace('\\+', ' ')
    result = korean_pattern.findall(text)
    return ''.join(result) if result else text

# Custom function to parse the date and time
def custom_datetime_parser(date_str):
    # Regular expression pattern to match date and time elements
    pattern = r'(\d{4})\. (\d{1,2})\. (\d{1,2})\. (\S+) (\d{1,2}:\d{2}:\d{2})'

    # Extract date and time components
    match = re.match(pattern, date_str)
    if match:
        year, month, day, am_pm, time = match.groups()
        hour, minute, second = map(int, time.split(':'))
        if am_pm == '오후' and hour < 12:
            hour += 12
        elif am_pm == '오전' and hour == 12:
            hour = 0
        return pd.Timestamp(year=int(year), month=int(month), day=int(day), hour=hour, minute=minute, second=second)
    else:
        return pd.NaT
    
    
def shift_values(row):
    cols = ['1', '2', '3', '4', '5']
    # Filter out None or NaN values
    filtered_values = [row[col] for col in cols if pd.notna(row[col])]
    # Shift values to the left
    for i, col in enumerate(cols):
        row[col] = filtered_values[i] if i < len(filtered_values) else None
    return row

def convert_column_name(col_name):
    if isinstance(col_name, str) and col_name.isdigit():
        return int(col_name)
    else:
        return col_name
    


# Function to determine the prefix based on "node"
def get_prefix(node):
    if node.startswith("https://blog.naver.com/hanpro911"):
        return "BH"
    elif node.startswith("https://blog.naver.com/mysecondplace"):
        return "BM"
    elif node.startswith("https://blog.naver.com"):
        return "BZ"
    elif node.startswith("https://cafe.naver.com/mysecondhouse1"):
        return "CM"
    elif node.startswith("https://mysecondplace.co.kr") and ("bmode=write" in node or "prod_code" in node):
        return "HA"
    elif node.startswith("https://mysecondplace.co.kr"):
        return "HH"
    elif node.startswith("https://maily.so"):
        return "HB"
    elif node.startswith("https://whattime.co.kr"):
        return "HC"
    elif node.startswith("https://m2place.imweb.me"):
        return "HO"
    elif node.startswith("https://instagram.com/mysecondplace_"):
        return "|M"
    elif node.startswith("http://pf.kakao.com/_vxkxngb"):
        return "KC"
    elif node.startswith("http"):
        return "SE"
    else:
        return "SK"
    
def get_last_number(column, prefix):
    last_number = 0
    for value in column.dropna():  # Exclude NaN values
        if str(value).startswith(prefix):
            last_number = max(last_number, int(str(value).split('-')[-1]))
    return last_number
    

# df_ga, df_blog의 규격화

## Blog

### node

In [8]:
# Filter rows based on whether contents_url starts with "http"
url_rows = df_blog[df_blog['before'].str.startswith('http')]
# Filter rows based on whether contents_url does not start with "http"
keyword_rows = df_blog[~df_blog['before'].str.startswith('http')]
# referer 구분하기
url_parts = url_rows['before'].apply(lambda x: re.split('proxyReferer|topReferer', x))
url_parts = url_parts.apply(lambda parts: [unquote(part.lstrip('=')) for part in parts])
url_parts = url_parts.apply(lambda parts: [unquote(part) for part in parts])
url_parts = url_parts.apply(lambda parts: [unquote(part) for part in parts])
url_parts = url_parts.apply(lambda parts: [unquote(part) for part in parts])
# Convert url_parts to a list
url_parts_list = url_parts.tolist()
# Create a new DataFrame with the referers
referers_df = pd.DataFrame(url_parts_list, columns=['url_before', 'url_referer_1', 'url_referer_2', 'url_referer_3'])
url_rows_reset = url_rows.reset_index(drop=True)
referers_df_reset = referers_df.reset_index(drop=True)

url_rows_2 = pd.concat([url_rows_reset, referers_df_reset], axis=1)
all_keys = set()

# Apply modifications to url_before, url_referer_1, url_referer_2, url_referer_3
for column in ['url_before', 'url_referer_1', 'url_referer_2', 'url_referer_3']:
    url_rows_2[f'modified_{column}'] = url_rows_2[column].apply(lambda x: extract_korean(x) if (isinstance(x, str) and any(c >= '가' and c <= '힣' for c in x)) else modify_url(x, all_keys))

# url을 정리한 값과 분리했었던 keyword_row 합치기
df_blog_2 = pd.concat([url_rows_2, keyword_rows],ignore_index=True)
df_blog_2['modified_url_before'] = df_blog_2['modified_url_before'].fillna(df_blog_2['before'])
df_blog_2['id'] = None
df_blog_2['id'] = df_blog_2.index + 1
df_blog_2['id'] = df_blog_2['id'].apply(lambda x: f'b-{x:06d}')

# Creating a new DataFrame 'df_acq' with selected columns from 'df_blog_standard'
df_blog_3 = df_blog_2[['id', 'date_info', 'before_count','node', 'modified_url_before', 'modified_url_referer_1', 'modified_url_referer_2', 'modified_url_referer_3']].copy()

# Renaming and reordering column
df_blog_3.rename(columns={
    'modified_url_referer_3': '1',
    'modified_url_referer_2': '2',
    'modified_url_referer_1': '3',
    'modified_url_before': '4',
    'node': '5'
}, inplace=True)

# before_count 만큼 행을 늘려라!
df_blog_3['before_count'] = df_blog_3['before_count'].astype(int).apply(lambda x: max(1, math.ceil(x)))

# page 순서대로 정리
df_blog_4 = df_blog_3.apply(shift_values, axis=1)
df_blog_4.dropna(axis=1, how='all', inplace=True)
df_blog_4 = df_blog_4.loc[df_blog_4.index.repeat(df_blog_4['before_count'])]

# 블로그 유입은 경로의 시작이 블로그가 아닌 아예 다른 곳인 경우만!
df_blog_5 = df_blog_4.copy()
df_blog_5.reset_index(drop=True, inplace=True)

# 'repeated_blog' DataFrame에서 '1'열의 값이 특정 조건을 만족하지 않는 행(= 시작 페이지가 블로그 페이지가 아닌)을 골라내자.
condition = (~df_blog_5['1'].str.startswith('https://blog.naver.com/mysecondplace')) & (~df_blog_5['1'].str.startswith('https://blog.naver.com/hanpro911'))
filtered_rows = df_blog_5[condition]

# '유입' 열에 해당 조건은 옮기고, 생긴 빈칸을 오른쪽에 있던 값으로 채움
df_blog_5.loc[condition, '유입'] = filtered_rows['1']
df_blog_5.loc[condition, '1'] = filtered_rows['2']
df_blog_5.loc[condition, '2'] = filtered_rows['3']
df_blog_5.loc[condition, '3'] = filtered_rows['4']
df_blog_5

# ga 데이터 형식의 컬럼을 추가
df_blog_5['소스'] = None
df_blog_5['매체'] = None
df_blog_5['캠페인'] = None
df_blog_5['컨텐츠'] = None
df_blog_5['검색어'] = None
df_blog_5['컨텐츠'] = None

# '유입' 데이터를 ga 형식으로 변환
df_blog_5['유입'] = df_blog_5['유입'].replace(np.nan, '')
df_blog_5.loc[pd.notna(df_blog_5['유입']) & df_blog_5['유입'].str.startswith('http'), '컨텐츠'] = df_blog_5['유입']
df_blog_5.loc[pd.notna(df_blog_5['유입']) & ~df_blog_5['유입'].str.startswith('http'), '검색어'] = df_blog_5['유입']
# 페이지 순서에 따른 열의 이름을 정리한다.
df_blog_5.columns = [convert_column_name(col) for col in df_blog_5.columns]

# 숫자 열 추출
numeric_columns = [col for col in df_blog_5.columns if isinstance(col, int)]
numeric_columns.sort()
# 열의 이름 변경
df_blog_5.rename(columns={
    '소스': 'source',
    '매체': 'medium',
    '캠페인': 'campaign',
    '컨텐츠': 'content',
    '검색어': 'keyword'
}, inplace=True)

# 최종 열의 순서와 필요한 것들만 추려서 df_blog_result에 저장
column_order = ['id', 'date_info', 'source', 'medium', 'campaign', 'content', 'keyword'] + numeric_columns
df_blog_result = df_blog_5[column_order].copy()
df_blog_result.columns = df_blog_result.columns.astype(str)
df_blog_result.dropna(how='all', inplace=True)




### Result_Blog

In [9]:
df_blog_result

,id,date_info,source,medium,campaign,content,keyword,1,2,3,4,5
0,b-000001,20240115,None,None,None,https://search.daum.net,None,https://blog.naver.com/mysecondplace/223208249149,None,None,None,None
1,b-000002,20240115,None,None,None,None,,https://blog.naver.com/mysecondplace/223208249149,https://blog.naver.com/mysecondplace/223212353603,None,None,None
2,b-000003,20240115,None,None,None,https://www.bing.com,None,https://blog.naver.com/mysecondplace/223241084754,None,None,None,None
3,b-000004,20240115,None,None,None,None,,https://blog.naver.com/mysecondplace/223289322164,https://blog.naver.com/mysecondplace/223258887581,None,None,None
4,b-000004,20240115,None,None,None,None,,https://blog.naver.com/mysecondplace/223289322164,https://blog.naver.com/mysecondplace/223258887581,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
2010,b-001567,20240213,None,None,None,None,화목 온천탕,https://blog.naver.com/hanpro911/222612556607,None,None,None,None
2011,b-001568,20240213,None,None,None,None,마이세컨플레이스,https://blog.naver.com/hanpro911/223319123073,None,None,None,None
2012,b-001569,20240213,None,None,None,None,세컨하우스 플랫폼,https://blog.naver.com/hanpro911/223319123073,None,None,None,None
2013,b-001570,20240213,None,None,None,None,공주 촌캉스 매매,https://blog.naver.com/hanpro911/222843016353,None,None,None,None


## GA

### 전처리

In [10]:
# 체류시간 0인 경우 채우기
df_ga['체류시간'].fillna(0, inplace=True)
# Apply the custom datetime parser to the '시간' column
df_ga['date_info'] = df_ga['시간'].apply(custom_datetime_parser)
# Format the 'date_info' column as 'yyyymmdd'
df_ga['date_info'] = df_ga['date_info'].dt.strftime('%Y%m%d')
# node_standarization
all_keys = set()
df_ga['modified_node']=df_ga['정규화링크'].apply(lambda x:modify_url(x, all_keys))

C:\Users\ch.kang\AppData\Local\Temp\ipykernel_25064\14379243.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_ga['체류시간'].fillna(0, inplace=True)


### admin_ids 추출하기

In [11]:

condition_a = df_ga['정규화링크'].str.contains('write|prod_code', case=False, regex=True)
condition_a = condition_a.fillna(False)  # NaN 값을 False로 대체
condition_a.value_counts()

filtered_a = df_ga[condition_a]

ids_a = filtered_a['아이디'].unique().tolist()  # 기존의 고유한 아이디 목록을 리스트로 변환
ids_a.append('1828185651.1705389498')  # 새로운 아이디를 추가

condition_b = df_ga.groupby('아이디')['체류시간'].sum() >= 18000
ids_b = condition_b[condition_b].index
admin_ids = set(ids_a) | set(ids_b)

print(len(ids_a), len(ids_b))
admin_ids

# 적용하려면 아래 코드를 주석처리하지 말고 실행하세요.
df_ga= df_ga[~df_ga['아이디'].isin(admin_ids)]


10 66


C:\Users\ch.kang\AppData\Local\Temp\ipykernel_25064\3828784831.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  condition_a = condition_a.fillna(False)  # NaN 값을 False로 대체


### node

In [12]:
df_ga_1 = df_ga[df_ga['이벤트명'] == 'page_view']

# grouped 객체 생성
grouped = df_ga_1.groupby(['아이디', 'date_info'])

# 가장 긴 그룹의 길이를 구함
max_length = grouped.size().max()

# 각 그룹에 대한 인덱스 생성
df_ga_1['dummy'] = grouped.cumcount() % max_length

# 컬럼 이름을 위한 새로운 컬럼 생성
df_ga_1['column_name'] = (grouped.cumcount() + 1).astype(str)

df_ga_2 = df_ga_1.pivot_table(
    index=['아이디', 'date_info'], 
    columns='column_name', 
    values='modified_node',
    aggfunc='first'
)

# Convert columns to float
df_ga_2.columns = df_ga_2.columns.astype(float)
# 컬럼 이름을 정수형으로 변환
df_ga_2.columns = df_ga_2.columns.astype(int)

# 컬럼을 숫자 순서대로 정렬
df_ga_2 = df_ga_2.sort_index(axis=1)
df_ga_2.sort_values(by='date_info', axis=0, inplace=True)
df_ga_2.reset_index(inplace=True)
def get_first_time(group):
    return group['시간'].iloc[0]
# Apply the function to the grouped object and reset the index
first_time_df = grouped.apply(get_first_time).reset_index()

df_ga_2 = pd.merge(df_ga_2, first_time_df, on=['아이디', 'date_info'], how='left', suffixes=('', '_first_time'))
df_ga_2.rename(columns={0: 'time'}, inplace=True)
df_ga_3 = df_ga[(df_ga['이벤트명'] == 'page_view') & df_ga['소스'].notna()]
df_ga_4 = pd.merge(df_ga_2, df_ga_3, left_on=['아이디', 'time', 'date_info'], right_on=['아이디', '시간', 'date_info'], how='left')
# 전환 여부를 묶어서 새로운 DataFrame 생성
df_ga_5 = df_ga[df_ga['전환여부'].notna()]
df_ga_6 = df_ga_5.groupby(['아이디', 'date_info'])['전환여부'].apply(lambda x: pd.Series(x.values)).unstack().reset_index()

# 새로운 열을 추가하여 최종 DataFrame 생성
num_conversions = df_ga_6.shape[1] - 2  # 전환여부 열을 제외한 열 개수
conversion_columns = [f'conversion_{i + 1}' for i in range(num_conversions)]
df_ga_6.columns = ['아이디', 'date_info'] + conversion_columns
# NaN 값 0으로 채우기
df_ga_6 = df_ga_6.fillna(0)

df_ga_7 = pd.merge(df_ga_4, df_ga_6, left_on=['아이디', 'date_info'], right_on=['아이디', 'date_info'], how='left')

# 숫자 열 추출
numeric_columns_ga = [col for col in df_ga_7.columns if isinstance(col, int)]
numeric_columns_ga.sort()
# 열의 이름 변경
df_ga_7.rename(columns={
    '아이디': 'id',
    '소스': 'source',
    '매체': 'medium',
    '캠페인': 'campaign',
    '컨텐츠': 'content',
    '검색어': 'keyword'
}, inplace=True)

# 최종 열의 순서와 필요한 것들만 추려서 df_blog_result에 저장
column_order_ga = ['id', 'date_info'] + conversion_columns + ['source', 'medium', 'campaign', 'content', 'keyword'] + numeric_columns_ga
df_ga_result = df_ga_7[column_order_ga].copy()
df_ga_result.columns = df_ga_result.columns.astype(str)
df_ga_result.dropna(how='all', inplace=True)
df_ga_result

C:\Users\ch.kang\AppData\Local\Temp\ipykernel_25064\1935871419.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ga_1['dummy'] = grouped.cumcount() % max_length
C:\Users\ch.kang\AppData\Local\Temp\ipykernel_25064\1935871419.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ga_1['column_name'] = (grouped.cumcount() + 1).astype(str)
C:\Users\ch.kang\AppData\Local\Temp\ipykernel_25064\1935871419.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is dep

,id,date_info,conversion_1,conversion_2,source,medium,campaign,content,keyword,1,...,50,51,52,53,54,55,56,57,58,59
0,1258902875.1704785609,20240115,NaN,NaN,cafe.naver.com,referral,(referral),NaN,NaN,https://mysecondplace.co.kr/secondhouse_blog/?...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,515817566.1705308322,20240115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://mysecondplace.co.kr,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,120896688.1705309550,20240115,NaN,NaN,fbad,class_3rd,after_b,Facebook_Right_Column,NaN,https://mysecondplace.co.kr/secondhouse_servic...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1650189843.1705312495,20240115,NaN,NaN,fbad,class_3rd,professional_w,Facebook_Mobile_Feed,NaN,https://mysecondplace.co.kr/secondhouse_servic...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,633531223.1697512712,20240115,NaN,NaN,igad,class_3rd,before_w,Instagram_Feed,NaN,https://mysecondplace.co.kr/secondhouse_servic...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3233,1875211093.1707724233,20240214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://mysecondplace.co.kr/secondhouse_service,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3234,552285055.1707887132,20240214,NaN,NaN,google,organic,(organic),NaN,(not provided),https://mysecondplace.co.kr/secondhouse_blog/?...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3235,1913223013.1707887891,20240214,NaN,NaN,google,organic,(organic),NaN,(not provided),https://mysecondplace.co.kr,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3236,1931554103.1707458495,20240214,NaN,NaN,newsletter,yangpyeong0214,yangpyeong0214,NaN,NaN,https://mysecondplace.co.kr/ya0008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Result_GA

In [13]:
df_ga_result

,id,date_info,conversion_1,conversion_2,source,medium,campaign,content,keyword,1,...,50,51,52,53,54,55,56,57,58,59
0,1258902875.1704785609,20240115,NaN,NaN,cafe.naver.com,referral,(referral),NaN,NaN,https://mysecondplace.co.kr/secondhouse_blog/?...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,515817566.1705308322,20240115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://mysecondplace.co.kr,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,120896688.1705309550,20240115,NaN,NaN,fbad,class_3rd,after_b,Facebook_Right_Column,NaN,https://mysecondplace.co.kr/secondhouse_servic...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1650189843.1705312495,20240115,NaN,NaN,fbad,class_3rd,professional_w,Facebook_Mobile_Feed,NaN,https://mysecondplace.co.kr/secondhouse_servic...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,633531223.1697512712,20240115,NaN,NaN,igad,class_3rd,before_w,Instagram_Feed,NaN,https://mysecondplace.co.kr/secondhouse_servic...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3233,1875211093.1707724233,20240214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://mysecondplace.co.kr/secondhouse_service,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3234,552285055.1707887132,20240214,NaN,NaN,google,organic,(organic),NaN,(not provided),https://mysecondplace.co.kr/secondhouse_blog/?...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3235,1913223013.1707887891,20240214,NaN,NaN,google,organic,(organic),NaN,(not provided),https://mysecondplace.co.kr,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3236,1931554103.1707458495,20240214,NaN,NaN,newsletter,yangpyeong0214,yangpyeong0214,NaN,NaN,https://mysecondplace.co.kr/ya0008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### GA + BLOG

In [14]:
df_result = pd.concat([df_ga_result, df_blog_result])

### Result_Blog+GA

In [15]:
df_result

,id,date_info,conversion_1,conversion_2,source,medium,campaign,content,keyword,1,...,50,51,52,53,54,55,56,57,58,59
0,1258902875.1704785609,20240115,NaN,NaN,cafe.naver.com,referral,(referral),NaN,NaN,https://mysecondplace.co.kr/secondhouse_blog/?...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,515817566.1705308322,20240115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://mysecondplace.co.kr,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,120896688.1705309550,20240115,NaN,NaN,fbad,class_3rd,after_b,Facebook_Right_Column,NaN,https://mysecondplace.co.kr/secondhouse_servic...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1650189843.1705312495,20240115,NaN,NaN,fbad,class_3rd,professional_w,Facebook_Mobile_Feed,NaN,https://mysecondplace.co.kr/secondhouse_servic...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,633531223.1697512712,20240115,NaN,NaN,igad,class_3rd,before_w,Instagram_Feed,NaN,https://mysecondplace.co.kr/secondhouse_servic...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010,b-001567,20240213,NaN,NaN,None,None,None,None,화목 온천탕,https://blog.naver.com/hanpro911/222612556607,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011,b-001568,20240213,NaN,NaN,None,None,None,None,마이세컨플레이스,https://blog.naver.com/hanpro911/223319123073,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012,b-001569,20240213,NaN,NaN,None,None,None,None,세컨하우스 플랫폼,https://blog.naver.com/hanpro911/223319123073,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013,b-001570,20240213,NaN,NaN,None,None,None,None,공주 촌캉스 매매,https://blog.naver.com/hanpro911/222843016353,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# 파일 경로 및 이름 생성
yesterday = datetime.now().date() - timedelta(days=1)
file_name = f"01_RawData_{yesterday}.xlsx"
full_file_path = os.path.join(upper_path, file_name)

# Excel 파일로 저장하면서 첫 번째 행 고정
with pd.ExcelWriter(full_file_path, engine='xlsxwriter') as writer:
    df_result.to_excel(writer, sheet_name='Sheet1', index=False)
    worksheet = writer.sheets['Sheet1']
    worksheet.freeze_panes(1, 0)  # 첫 번째 행을 고정

# conversion

### inflow -> 02_Inflow_Statistics_{yesterday}.xlsx"

In [27]:
count_hh = df_ga_result.shape[0]
count_bm_hh = len(df_ga_result[df_ga_result['content'].str.startswith('https://blog.naver.com/mysecondplace').fillna(False)])
count_bh_hh = len(df_ga_result[df_ga_result['content'].str.startswith('https://blog.naver.com/hanpro911').fillna(False)])
count_ia_hh = len(df_ga_result[df_ga_result['source'].str.startswith('ig').fillna(False)])
count_fa_hh = len(df_ga_result[df_ga_result['source'].str.startswith('fb').fillna(False)])
count_ih_hh = len(df_ga_result[df_ga_result['source'].str.startswith('linktr.ee').fillna(False)])
count_kw_hh = count_hh - df_ga_result['keyword'].isna().sum()
count_ir_hh = len(df_ga_result[df_ga_result['source'].str.startswith('igreels').fillna(False)])
count_yt_hh = len(df_ga_result[df_ga_result['source'].str.startswith('youtube').fillna(False)])
count_sy_hh = len(df_ga_result[df_ga_result['source'].str.startswith('youtube.com').fillna(False)])

count_bm = len(df_blog_result[df_blog_result['1'].str.startswith('https://blog.naver.com/mysecondplace')])
count_bh = len(df_blog_result[df_blog_result['1'].str.startswith('https://blog.naver.com/hanpro911')])

counts = {
    '홈페이지 방문수': count_hh,
    '마세플 블로그 방문수': count_bm,
    '마세플 블로그 보고 홈페이지 방문수': count_bm_hh,
    '한프로 블로그 방문수': count_bh,
    '한프로 블로그 보고 홈페이지 방문수': count_bh_hh,
    '인스타광고(릴스 광고 포함) 보고 홈페이지 방문수': count_ia_hh,
    '인스타 릴스 광고 보고 홈페이지 방문수': count_ir_hh,
    '페북광고 보고 홈페이지 방문수': count_fa_hh,
    '인스타 홈 보고 홈페이지 방문수': count_ih_hh,
    '검색어로 홈페이지 방문수': count_kw_hh,
    '유튜브에서 홈페이지 방문수': count_yt_hh,
    '유튜브 동영상 보고 홈페이지 방문수': count_yt_hh - count_sy_hh

}

# Create DataFrame
inflow_statistics = pd.DataFrame(list(counts.items()), columns=['Category', 'Count'])
file_name = f"02_Inflow_Statistics_{yesterday}.xlsx"
full_file_path = os.path.join(upper_path, file_name)
inflow_statistics.to_excel(full_file_path, index=False)
inflow_statistics


C:\Users\ch.kang\AppData\Local\Temp\ipykernel_25064\2631361523.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  count_bm_hh = len(df_ga_result[df_ga_result['content'].str.startswith('https://blog.naver.com/mysecondplace').fillna(False)])
C:\Users\ch.kang\AppData\Local\Temp\ipykernel_25064\2631361523.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  count_bh_hh = len(df_ga_result[df_ga_result['content'].str.startswith('https://blog.naver.com/hanpro911').fillna(False)])
C:\Users\ch.kang\AppData\Local\Temp\ipykernel_25064\2631361523.py:4: FutureWarning

,Category,Count
0,홈페이지 방문수,3238
1,마세플 블로그 방문수,1219
2,마세플 블로그 보고 홈페이지 방문수,61
3,한프로 블로그 방문수,681
4,한프로 블로그 보고 홈페이지 방문수,31
5,인스타광고(릴스 광고 포함) 보고 홈페이지 방문수,1003
6,인스타 릴스 광고 보고 홈페이지 방문수,333
7,페북광고 보고 홈페이지 방문수,309
8,인스타 홈 보고 홈페이지 방문수,100
9,검색어로 홈페이지 방문수,495


### Keyword -> 02_Inflow_Keyword_{yesterday}.xlsx

In [18]:
# Filter rows where 'keyword' is not "(not provided)"
keywords_hh = df_ga_result[df_ga_result['keyword'] != '(not provided)']
keyword_hh = pd.DataFrame({'Keyword_Homepage': keywords_hh['keyword'].value_counts().index, 'Count_Homepage': keywords_hh['keyword'].value_counts().values})
keyword_hh

,Keyword_Homepage,Count_Homepage
0,120206410183050614,144
1,마이세컨플레이스,64
2,마이세컨하우스,6
3,마이 세컨 플레이스,3
4,클리 세컨하우스,3
...,...,...
68,세컨하우스 매물,1
69,미등기시골집리모델링시허가사항,1
70,시골집 소형 세컨하우스,1
71,세컨하우스 관리,1


In [19]:
# Create a copy of the DataFrame to avoid the warning
keywords_bm = df_blog_result[df_blog_result['1'].str.startswith("https://blog.naver.com/mysecondplace")].copy()
keywords_bh = df_blog_result[df_blog_result['1'].str.startswith("https://blog.naver.com/hanpro911")].copy()

# Strip whitespaces from the 'keyword' column
keywords_bm['keyword'] = keywords_bm['keyword'].str.strip()
keywords_bm = keywords_bm[keywords_bm['keyword'] != '']
keywords_bh['keyword'] = keywords_bh['keyword'].str.strip()
keywords_bh = keywords_bh[keywords_bh['keyword'] != '']


# For keywords_bm
keyword_bm = pd.DataFrame({'Keyword_blogMSP': keywords_bm.dropna(subset=['keyword'])['keyword'].value_counts().index,
                           'Count_blogMSP': keywords_bm.dropna(subset=['keyword'])['keyword'].value_counts().values})
# For keywords_bm
keyword_bh = pd.DataFrame({'Keyword_blogHAN': keywords_bh.dropna(subset=['keyword'])['keyword'].value_counts().index,
                           'Count_blogHAN': keywords_bh.dropna(subset=['keyword'])['keyword'].value_counts().values})

In [20]:
keyword_bm.head(20)

,Keyword_blogMSP,Count_blogMSP
0,세컨 하우스란,74
1,공주시골집매매,37
2,공주 시골집 매매,27
3,세컨하우스,24
4,공주 시골집매매,22
5,시골집 구매,18
6,화목 노천탕,16
7,릴호스,15
8,미등기 시골주택 등기 방법,13
9,시골집 양성화,12


In [21]:
keyword_bh.head(20)

,Keyword_blogHAN,Count_blogHAN
0,노천탕 만들기,51
1,화목 노천탕,20
2,경기도 빈집매매,17
3,시골집 세컨하우스,10
4,마이세컨플레이스,9
5,세컨하우스 노천탕,7
6,공주시골집매매,7
7,시골 세컨하우스,7
8,꺽지구이,6
9,노천탕만들기,5


In [22]:
inflow_keyword = pd.concat([keyword_bh, keyword_bm, keyword_hh], axis = 1)

In [23]:
file_name = f"02_Inflow_Keyword_{yesterday}.xlsx"
full_file_path = os.path.join(upper_path, file_name)
inflow_keyword.to_excel(full_file_path, index=False)

### Conversion in HH -> 02_Conversion_HomepageData_{yesterday}.xlsx

In [24]:
conversion_df = df_ga_result[df_ga_result['conversion_1'].notna()]
conversion_ids = conversion_df['id']

# Corrected typo in the next line
df_conversion = df_ga[df_ga['아이디'].isin(conversion_ids)]
file_name = f"02_Conversion_HomepageData_{yesterday}.xlsx"
full_file_path = os.path.join(upper_path, file_name)
df_conversion.to_excel(full_file_path, index=False)

# index for conversion
selected_columns = ['id'] + [col for col in conversion_df.columns if col.startswith('conversion')] + ['date_info']
# Use loc to select the specified columns and their corresponding index values
index_conversion = conversion_df.loc[:, selected_columns]
index_conversion


,id,conversion_1,conversion_2,date_info
206,429492541.1705458222,장바구니,0,20240117
618,1091021041.1699653737,장바구니,0,20240119
692,719573913.1705709007,장바구니,0,20240120
1096,1837415535.1705999129,서비스구매,0,20240123
1192,322917954.1705984434,서비스구매,0,20240123
1288,876186389.1706079428,회원가입,0,20240124
1409,1159656766.1706252146,회원가입,0,20240126
1520,1767893859.1706580292,회원가입,서비스구매,20240130
1570,709216321.1706669384,장바구니,0,20240131
1927,864564312.1699098946,회원가입,0,20240206


In [25]:
# 특정한 id의 전체 행동을 추출
## 특정한 id 선택
### df_result에서 source가 igreels인 경우

igreels_ids_sorted = sorted(df_result[df_result['source'] == 'igreels']['id'].tolist())
print(len(igreels_ids_sorted), "개의 id")
igreels_ids_sorted


333 개의 id


['1014877735.1707668021',
 '1019278131.1707628757',
 '1023018948.1707299466',
 '1033168662.1707472712',
 '1033168662.1707472712',
 '1041664988.1706015221',
 '1050828352.1707741826',
 '1051547271.1707665009',
 '1062519455.1707734369',
 '1062567649.1707205585',
 '1065164114.1697873403',
 '107505261.1707220511',
 '1079869378.1707544719',
 '108758320.1707326743',
 '1089509498.1692364455',
 '1090340132.1707122317',
 '1090340132.1707122317',
 '1090340132.1707122317',
 '1090340132.1707122317',
 '1090340132.1707122317',
 '1094557665.1707205274',
 '1100398369.1693746857',
 '1102643303.1676883758',
 '1107912774.1707746003',
 '1121898591.1707544023',
 '1122663159.1707266147',
 '1130239469.1707196301',
 '1135692901.1694234386',
 '1146974630.1707401948',
 '1147238901.1707210025',
 '11540413.1707711089',
 '1154804613.1707311680',
 '115601215.1707645279',
 '1159793807.1707546877',
 '1159793807.1707546877',
 '1160357143.1697615166',
 '1162205010.1707479005',
 '1172427172.1707472749',
 '1183123087.1707

In [26]:
## 특정한 id의 행동 추출

### 특정한 id의 중복 제거
unique_ids = list(set(igreels_ids_sorted)) #릴스 보고 들어온 id
### 중복 제거한 특정한 id의 행동을 원본 ga 데이터(df_ga)에서 추출
behavior_of_specific_id = df_ga[df_ga['아이디'].isin(unique_ids)]

today = datetime.now().date()
file_name = f"99_igreels_{today}.xlsx"
full_file_path = os.path.join(upper_path, file_name)
behavior_of_specific_id.to_excel(full_file_path, index=False)

### conversion_depth -> "02_conversion_depth_{yesterday}.xlsx"

file_name = f"02_conversion_depth_{yesterday}.xlsx"
full_file_path = os.path.join(upper_path, file_name)
conversion_depth.to_excel(full_file_path, index=False)

# Specify multiple values to check
specific_values = ['https://mysecondplace.co.kr/login', 'another_value', 'yet_another_value']

# Count occurrences of specific values in the 'content' column
count_specific_values = df_ga_result['content'].isin(specific_values).sum()


# Node - Edge

G = nx.Graph()




# 노드 추가 및 속성 설정
for index, row in df.iterrows():
    node_id = row['node']
    node_label = row[0]
    node_attributes = {key: row[key] for key in df.columns[10:]}  # 노드의 투명도와 다른 속성들을 딕셔너리로 저장
    G.add_node(node_id, label=node_label, **node_attributes)

# 노드 및 간선 정보
nodes = list(unique_nodes)
edges = []

for _, row in referers_df.iterrows():
    source_node = row['modified_url_before']
    target_node = row[df.columns[1]]  # 두 번째 열의 값을 노드로 사용
    weight = row['view']

    # 중복된 노드 방지를 위해 노드 추가 여부 확인
    if source_node not in unique_nodes:
        unique_nodes.add(source_node)

    if target_node not in unique_nodes:
        unique_nodes.add(target_node)

    # 간선 추가
    edges.append((source_node, target_node, {"weight": weight}))

    # modified_url_referer_1 열 추가
    if not pd.isnull(row['modified_url_referer_1']):
        source_node = row['modified_url_referer_1']
        unique_nodes.add(source_node)
        edges.append((source_node, target_node, {"weight": weight}))

        # modified_url_referer_2 열 추가
        if not pd.isnull(row['modified_url_referer_2']):
            source_node = row['modified_url_referer_2']
            unique_nodes.add(source_node)
            edges.append((source_node, target_node, {"weight": weight}))

            # modified_url_referer_3 열 추가
            if not pd.isnull(row['modified_url_referer_3']):
                source_node = row['modified_url_referer_3']
                unique_nodes.add(source_node)
                edges.append((source_node, target_node, {"weight": weight}))



# Assuming 'node' is a column in your DataFrame and '0' is the first column
node_to_label_mapping = dict(zip(df['node'], df.iloc[:, 0]))

# Now, you can use this mapping in NetworkX
G = nx.Graph()
G.add_nodes_from(df['node'])
nx.set_node_attributes(G, node_to_label_mapping, 'label')

# Find and remove edges with weight 0
zero_weight_edges = [(u, v) for u, v, d in G.edges(data=True) if d['weight'] == 0]
G.remove_edges_from(zero_weight_edges)     
# Find isolated nodes
isolated_nodes = list(nx.isolates(G))
# Remove isolated nodes from the graph
G.remove_nodes_from(isolated_nodes)
# Remove self-loops
G.remove_edges_from(nx.selfloop_edges(G))



plt.figure(figsize=(23.4, 33.1))  # A2 paper size (in inches)
        



# NODE SETTING

# Calculate node sizes based on total views
view_df['total_views'] = pd.to_numeric(df['total_views'], errors='coerce')
view_df = view_df.dropna(subset=['total_views'])
default_node_size = 100  # Default node size (used when there are no views)
max_views = view_df['total_views'].max()  # Calculate the maximum views
node_sizes = {url: 5000 * (views / max_views) if url in view_df['contents_url'].values else default_node_size for url, views in view_df[['contents_url', 'total_views']].values}

pos = nx.fruchterman_reingold_layout(G, k=3, iterations = 600)
# pos = nx.kamada_kawai_layout(G)
# pos = nx.shell_layout(G)

# Draw nodes with labels
node_labels = {node: label for node, label in nx.get_node_attributes(G, 'label').items()}
nx.draw_networkx_labels(G, pos, labels=node_labels, font_size=10)
node_sizes_to_draw = [node_sizes.get(node, default_node_size) for node in G.nodes()]
nx.draw_networkx_nodes(G, pos, node_size=node_sizes_to_draw, node_color="lightblue")



# EDGE SETTING

if G.edges():
    min_weight = min([G[u][v]['weight'] for u, v in G.edges()])
    max_weight = max([G[u][v]['weight'] for u, v in G.edges()])
    scaling_factor_width = 10

    # Create a list of edge widths based on weights
    edge_widths = [scaling_factor_width * (G[u][v]['weight'] - min_weight) / (max_weight - min_weight) for u, v in G.edges()]

    # Draw edges with specified widths and arrows
    edges = nx.draw_networkx_edges(G, pos, arrowstyle="->", arrowsize=10, width=edge_widths, edge_color='g', arrows=True, connectionstyle='arc3,rad=0.1')

    # Set edge transparency based on edge weights
    scaling_factor_alpha = 10
    for i, (u, v, d) in enumerate(G.edges(data=True)):
        weight_normalized = (d['weight'] - min_weight) / (max_weight - min_weight)
        weight_alpha = scaling_factor_alpha * weight_normalized
        edges[i].set_alpha(min(1, weight_alpha))  # Limit maximum alpha to 1



# Display the graph
# Save the graph as an image file (e.g., PNG)
plt.savefig('..//PageRelation.png', format='png', dpi = 600)

plt.axis('off')
plt.show()